<a href="https://colab.research.google.com/github/SaraFernandes98/Fire-Detection---Firefront/blob/main/Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import os,sys
import cv2
import matplotlib.image as mpimg
import imageio
import pylab
import glob

from sklearn.metrics import accuracy_score, precision_score, recall_score, average_precision_score, confusion_matrix, f1_score
from sklearn.model_selection import train_test_split, KFold
from tensorflow import Tensor
from tensorflow.keras import layers, losses, backend
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import Sequence
from sklearn.utils import shuffle
from sklearn.preprocessing import OneHotEncoder
from keras.callbacks import LearningRateScheduler
from sklearn.utils import class_weight
from tqdm import tqdm
from keras.models import load_model

from math import floor
import random
import math
import imutils
from statistics import median
from matplotlib import transforms


# Random seed fixation
tf.random.set_seed(666)
np.random.seed(666)


Load images

In [8]:
def load_images_ssl_tain(path):
  images = []
  for filename in os.listdir(path):
      image = tf.io.read_file(path+'/'+filename)
      image = tf.image.decode_jpeg(image, channels=3)
      image = tf.image.convert_image_dtype(image, tf.float32)
      if image is not None:
          images.append(image)
  return np.array(images)

def load_images_supervised(path):
  data = pd.read_csv(path, iterator=True)
  dir = path.split('.')
  dataset= list(data)
  dataset = np.array(dataset)
  images=[]
  labels = []
  img_paths = []
  for i in dataset:
    for a in i:
      if os.path.exists(path[:-9]+a[0]):
        image_string = tf.io.read_file(path[:-9]+a[0])
        image = tf.image.decode_png(image_string, channels=3)
        image = tf.image.convert_image_dtype(image, tf.float32)
        label = a[1]
        images.append(image)
        labels.append(label)

  return images, labels

Pssl = '/content/drive/MyDrive/Tese/Bombeiros/All_crop_resized/to_train_ssl'
ssl_images = load_images_ssl_tain(Pssl)

Psupervised = '/content/drive/MyDrive/Tese/Bombeiros/All_crop_resized/supervised/labels.csv'
sup_images, sup_labels = load_images_supervised(Psupervised)


### **SSL Phase**



Autoencoder

In [5]:
def encoderModel():
  model = tf.keras.Sequential()
  #------Encoder------#
  model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
  model.add(layers.AveragePooling2D((2, 2), padding='same'))
  model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
  model.add(layers.AveragePooling2D((2, 2), padding='same'))
  model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
  model.add(layers.AveragePooling2D((2, 2), padding='same'))
  model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
  model.add(layers.AveragePooling2D((2, 2), padding='same'))
  model.add(layers.Conv2D(64, (3, 3), activation='tanh', padding='same'))
  model.add(layers.AveragePooling2D((2, 2), padding='same'))
  
  #------Decoder------#
  model.add(layers.Conv2DTranspose(64, (3, 3), activation='relu', strides=2, padding='same')) 
  model.add(layers.Conv2DTranspose(64, (3, 3), activation='relu', strides=2, padding='same'))
  model.add(layers.Conv2DTranspose(64, (3, 3), activation='relu', strides=2, padding='same'))
  model.add(layers.Conv2DTranspose(32, (3, 3), activation='relu', strides=2, padding='same'))
  model.add(layers.Conv2DTranspose(32, (3, 3), activation='relu', strides=2, padding='same'))
  model.add(layers.Conv2D(3, (3, 3), activation='relu', padding='same'))
  return model

In [3]:
#For autoencoder
class ColorAugment(object):
    def __call__(self, sample, p=1):        
        # Random flips
        #sample = self._random_apply(tf.image.flip_left_right, sample, p=0.5)
        
        # Randomly apply transformation (color distortions) with probability p.
        sample = self._random_apply(self._color_jitter, sample, p=1)
        return sample

    def _color_jitter(self, x,s=1):
        # one can also shuffle the order of following augmentations
        # each time they are applied.
        imgs = []
        for i in x:
          i = tf.image.random_saturation(i, lower=1-0.3*s, upper=1+0.3*s)
          i = tf.image.random_hue(i, max_delta=0.2*s)
          i = tf.clip_by_value(i, 0, 1)
          imgs.append(i)
        return np.array(imgs)
    

    def _random_apply(self, func, x, p):
        return tf.cond(
          tf.less(tf.random.uniform([], minval=0, maxval=1, dtype=tf.float32),
                  tf.cast(p, tf.float32)),
          lambda: func(x),
          lambda: x)

class CropAugment(object):
   def __call__(self, sample,p=1):        
        sample = self._random_apply(self._random_crop, sample, p=1)
        return sample

   def _random_crop(self,x):
      imgs = []
      for i in x:
            i = tf.image.random_flip_left_right(i)
            size = np.random.randint(100,200)
            i = tf.image.random_crop(i, [size,size,3])
            i = tf.image.resize(i,[224,224])
            imgs.append(i)
      return np.array(imgs)

   def _random_apply(self, func, x, p):
        return tf.cond(
          tf.less(tf.random.uniform([], minval=0, maxval=1, dtype=tf.float32),
                  tf.cast(p, tf.float32)),
          lambda: func(x),
          lambda: x)


class RotationAugment(object):
   def __call__(self, sample,p=1):        
        sample = self._random_apply(self._random_rotation, sample, p=1)
        return sample

   def _rotatedRectWithMaxArea(self, w, h, angle):
      """
      Given a rectangle of size wxh that has been rotated by 'angle' (in
      radians), computes the width and height of the largest possible
      axis-aligned rectangle (maximal area) within the rotated rectangle.
      """
      if w <= 0 or h <= 0:
        return 0,0

      width_is_longer = w >= h
      side_long, side_short = (w,h) if width_is_longer else (h,w)

      # since the solutions for angle, -angle and 180-angle are all the same,
      # if suffices to look at the first quadrant and the absolute values of sin,cos:
      sin_a, cos_a = abs(math.sin(angle)), abs(math.cos(angle))
      if side_short <= 2.*sin_a*cos_a*side_long or abs(sin_a-cos_a) < 1e-10:
        # half constrained case: two crop corners touch the longer side,
        #   the other two corners are on the mid-line parallel to the longer line
        x = 0.5*side_short
        wr,hr = (x/sin_a,x/cos_a) if width_is_longer else (x/cos_a,x/sin_a)
      else:
        # fully constrained case: crop touches all 4 sides
        cos_2a = cos_a*cos_a - sin_a*sin_a
        wr,hr = (w*cos_a - h*sin_a)/cos_2a, (h*cos_a - w*sin_a)/cos_2a

      return wr,hr

   def _random_rotation(self,x):
      imgs = []
      for i in x:
        angle = np.random.randint(1,20)
        wr, hr = self._rotatedRectWithMaxArea(224,224,math.radians(angle))
        # Use PIL or other library of the sort to rotate
        i = imutils.rotate(np.array(i), angle=angle)
        i = tf.image.central_crop(i, float(hr)/224) 
        i = tf.image.resize(i, [224,224])
        imgs.append(i)
      return np.array(imgs)
      #imgs.append(i)
      #return np.array(imgs)

   def _random_apply(self, func, x, p):
        return tf.cond(
          tf.less(tf.random.uniform([], minval=0, maxval=1, dtype=tf.float32),
                  tf.cast(p, tf.float32)),
          lambda: func(x),
          lambda: x)

class dataTransforms(object):
  def __call__(self, sample): 
        # Build the augmentation pipeline
        color_augmentation = ColorAugment()
        rotation_augmentation = RotationAugment()
        sample = self._random_apply(rotation_augmentation._random_rotation, sample, p=1)
        sample = self._random_apply(color_augmentation._color_jitter, sample, p=1)
        return sample

  def _random_apply(self, func, x, p):
        return tf.cond(
          tf.less(tf.random.uniform([], minval=0, maxval=1, dtype=tf.float32),
                  tf.cast(p, tf.float32)),
          lambda: func(x),
          lambda: x)
        


# Build the augmentation pipeline
color_augmentation = tf.keras.Sequential([layers.Lambda(ColorAugment())])
crop_augmentation = tf.keras.Sequential([layers.Lambda(CropAugment())])
rotation_augmentation = tf.keras.Sequential([layers.Lambda(RotationAugment())])
transformation = tf.keras.Sequential([layers.Lambda(dataTransforms())])


In [6]:
def dataAugmentation(dataset, nImgs, transform, per):
  """
    dataset: set with images
    nImgs: initial images (without transformations) used to train the network 
    transform: type of transformation to apply to images
    per: transformations are applied to a percentage of nImgs

    The total number of images used to train the network is nImgs*(1+per) 

    retrun: final images with transformations applied

  """
  train_auto_imgs = []
  train_auto_imgs = dataset[:nImgs]
  print(len(train_auto_imgs))
  if transform == 'color':
    train_auto_imgs = np.concatenate((train_auto_imgs, color_augmentation(dataset[:floor(nImgs*per)])))
  elif transform == 'rot':
    train_auto_imgs = np.concatenate((train_auto_imgs, rotation_augmentation(dataset[:floor(nImgs*per)])))
  elif transform == 'cor_rot':
    train_auto_imgs = np.concatenate((train_auto_imgs, color_augmentation(dataset[:floor(nImgs*per*0.5)])))
    train_auto_imgs = np.concatenate((train_auto_imgs, rotation_augmentation(dataset[floor(nImgs*per*0.5):floor(nImgs*per)])))
  elif transform == 'cor+rot':
    train_auto_imgs = np.concatenate((train_auto_imgs, transformation(dataset[:floor(nImgs*per)])))
  elif transform == '4x':
    train_auto_imgs = np.concatenate((train_auto_imgs, color_augmentation(dataset[:floor(nImgs*per)])))
    train_auto_imgs = np.concatenate((train_auto_imgs, rotation_augmentation(dataset[:floor(nImgs*per)])))
    train_auto_imgs = np.concatenate((train_auto_imgs, transformation(dataset[:floor(nImgs*per)])))
  elif transform == 'nocor':
    train_auto_imgs = np.concatenate((train_auto_imgs, rotation_augmentation(dataset[:floor(nImgs*per)])))
    train_auto_imgs = np.concatenate((train_auto_imgs, crop_augmentation(dataset[:floor(nImgs*per)])))
    train_auto_imgs = np.concatenate((train_auto_imgs, crop_augmentation(rotation_augmentation(dataset[:floor(nImgs*per)]))))
  elif transform == 'crop+flip':
    train_auto_imgs = np.concatenate((train_auto_imgs, crop_augmentation(dataset[:floor(nImgs*per)])))
  elif transform == '5x':
    train_auto_imgs = np.concatenate((train_auto_imgs, color_augmentation(dataset[:floor(nImgs*per)])))
    train_auto_imgs = np.concatenate((train_auto_imgs, rotation_augmentation(dataset[:floor(nImgs*per)])))
    train_auto_imgs = np.concatenate((train_auto_imgs, transformation(dataset[:floor(nImgs*per)])))
    train_auto_imgs = np.concatenate((train_auto_imgs, crop_augmentation(dataset[:floor(nImgs*per)])))
  
  return np.array(train_auto_imgs)

In [ ]:
#Get model
autoencoder = encoderModel()
autoencoder.compile(
    optimizer='adam',
    loss=losses.MeanSquaredError()
)

#Apply transformation to the images
images = dataAugmentation(ssl_images, 1000, 'color', 1) 
images = shuffle(images)
train, val = images[:floor(len(images)*0.9)], images[floor(len(images)*0.9):]

#train images
history = autoencoder.fit(train, train,   
                  epochs=60,
                  batch_size=75,
                  validation_data =(val, val), 
                  callbacks=[],
                  shuffle=True)


# Save the model.
model_name = 'model_name'
Pmodel = '/content/drive/MyDrive/Tese/code/Models/Self_supervised/' + model_name 
autoencoder.save(Pmodel)

SimCLR

In [5]:
#-----------------------Model-------------------#

def create_encoder():
    model = tf.keras.Sequential()
    # #------Encoder------#
    model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
    model.add(layers.AveragePooling2D((2, 2), padding='same'))
    model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
    model.add(layers.AveragePooling2D((2, 2), padding='same'))
    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(layers.AveragePooling2D((2, 2), padding='same'))
    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(layers.AveragePooling2D((2, 2), padding='same'))
    model.add(layers.Conv2D(64, (3, 3), activation='tanh', padding='same'))
    model.add(layers.AveragePooling2D((2, 2), padding='same'))
    
    return model


#Projection head
def add_projection_head(encoder):
    inputs = tf.keras.Input(shape=(224,224,3))
    for layer in encoder.layers:
        layer.trainable = True
    features = encoder(inputs)
    features = layers.Flatten()(features)
    projection_0 = layers.Dense(64)(features)
    projection_0 = layers.Activation("relu")(projection_0)
    projection_1 = layers.Dense(64)(projection_0)


    model = Model(inputs, projection_1)

    return model

In [10]:
#---------------Data Augmentation-------------------#
def random_resize_crop(image, crop_size=224):
    imgs=[]
    i=image[0]
    for i in image:
      print(i)
      size = np.random.randint(100,200)
      i = tf.image.random_crop(i, [size,size,3])
      i = tf.image.resize(i,[crop_size,crop_size])
      imgs.append(i)
    return np.array(i)

def flip_random_crop(image):
    image = tf.image.random_flip_left_right(image)
    image = random_resize_crop(image)
    return image


def color_jitter(x, strength=1):    
    x = tf.image.random_saturation(x, lower=1 - 0.3 * strength, upper=1 + 0.3 * strength)
    x = tf.image.random_hue(x, max_delta=0.2 * strength)
    x = tf.clip_by_value(x, 0, 1)
    return x



from PIL import Image
def _rotatedRectWithMaxArea( w, h, angle):
      """
      Given a rectangle of size wxh that has been rotated by 'angle' (in
      radians), computes the width and height of the largest possible
      axis-aligned rectangle (maximal area) within the rotated rectangle.
      """
      if w <= 0 or h <= 0:
        return 0,0

      width_is_longer = w >= h
      side_long, side_short = (w,h) if width_is_longer else (h,w)

      # since the solutions for angle, -angle and 180-angle are all the same,
      # if suffices to look at the first quadrant and the absolute values of sin,cos:
      sin_a, cos_a = abs(math.sin(angle)), abs(math.cos(angle))
      if side_short <= 2.*sin_a*cos_a*side_long or abs(sin_a-cos_a) < 1e-10:
        # half constrained case: two crop corners touch the longer side,
        #   the other two corners are on the mid-line parallel to the longer line
        x = 0.5*side_short
        wr,hr = (x/sin_a,x/cos_a) if width_is_longer else (x/cos_a,x/sin_a)
      else:
        # fully constrained case: crop touches all 4 sides
        cos_2a = cos_a*cos_a - sin_a*sin_a
        wr,hr = (w*cos_a - h*sin_a)/cos_2a, (h*cos_a - w*sin_a)/cos_2a

      return wr,hr

def random_rotation(x):
    imgs = []
    for i in x:
        angle = np.random.randint(1,20)
        wr, hr = _rotatedRectWithMaxArea(224,224,math.radians(angle))
        # Use PIL or other library of the sort to rotate
        i = imutils.rotate(np.array(i), angle=angle)
        i = tf.image.central_crop(i, float(hr)/224) 
        i = tf.image.resize(i, [224,224])
        imgs.append(i)
    return np.array(imgs)


def random_apply(func, x, p):
    if tf.random.uniform([], minval=0, maxval=1) < p:
        return func(x)
    else:
        return x




In [ ]:
!wget https://raw.githubusercontent.com/sthalles/SimCLR-tensorflow/master/utils/helpers.py
!wget https://raw.githubusercontent.com/sthalles/SimCLR-tensorflow/master/utils/losses.py

In [13]:
BATCH_SIZE = 128

@tf.function
def parse_images(image):
  return image

@tf.function
def train_step(xis, xjs, model, optimizer, temperature):
    with tf.GradientTape() as tape:
        zis = model(xis)
        zjs = model(xjs)
        # normalize projection feature vectors
        #em vez de normalizar aqui
        zis = tf.math.l2_normalize(zis, axis=1)
        zjs = tf.math.l2_normalize(zjs, axis=1)

        z = tf.cast(tf.concat((zis, zjs), 0), dtype=tf.float32)
        loss = 0
        for k in range(BATCH_SIZE):
            # Numerator (compare i,j & j,i)
            i = k
            j = k + BATCH_SIZE
            # Instantiate the cosine similarity loss function
            cosine_sim = tf.keras.losses.CosineSimilarity(axis=-1, reduction=tf.keras.losses.Reduction.NONE)
            sim = tf.squeeze(- cosine_sim(tf.reshape(z[i], (1, -1)), tf.reshape(z[j], (1, -1))))
            numerator = tf.math.exp(sim / temperature)

            # Denominator (compare i & j to all samples apart from themselves)
            sim_ik = - cosine_sim(tf.reshape(z[i], (1, -1)), z[tf.range(BATCH_SIZE*2) != i])
            sim_jk = - cosine_sim(tf.reshape(z[j], (1, -1)), z[tf.range(BATCH_SIZE*2) != j])
            denominator_ik = tf.reduce_sum(tf.math.exp(sim_ik / temperature))
            denominator_jk = tf.reduce_sum(tf.math.exp(sim_jk / temperature))

            # Calculate individual and combined losses
            loss_ij = - tf.math.log(numerator / denominator_ik)
            loss_ji = - tf.math.log(numerator / denominator_jk)
            loss += loss_ij + loss_ji
        # Divide by the total number of samples
        loss /= (BATCH_SIZE*2)


    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    return loss

def train_simclr(model, dataset, optimizer, criterion,
                 temperature=0.1, epochs=100):
    step_wise_loss = []
    epoch_wise_loss = []

    for epoch in tqdm(range(epochs)):
        for image_batch in dataset:
            #data augmentation: create positive and negative pairs
            a = random_apply(random_rotation,image_batch, p=1)
            b = image_batch 
            #train model and calculate loss
            loss = train_step(a, b, model, optimizer, temperature)
            step_wise_loss.append(loss)

        epoch_wise_loss.append(np.mean(step_wise_loss))
        #wandb.log({"nt_xentloss": np.mean(step_wise_loss)})
        
        if epoch % 1 == 0:
            print("step_wise_loss: ", step_wise_loss)
            print("epoch: {} loss: {:.3f}".format(epoch + 1, np.mean(step_wise_loss)))

    return epoch_wise_loss

In [ ]:
criterion = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, 
                                                          reduction=tf.keras.losses.Reduction.SUM)

decay_steps = 50 
lr_decayed_fn = tf.keras.experimental.CosineDecay(
    initial_learning_rate=0.01, decay_steps=decay_steps)
optimizer = tf.keras.optimizers.Adam(lr_decayed_fn)

#create model encoder + projection head
encoder = create_encoder()
encoder_with_projection_head = add_projection_head(encoder) 
encoder_with_projection_head.compile(
    optimizer=optimizer,
    loss=losses.MeanSquaredError())


#Get transformations to apply to images
train_ds = tf.data.Dataset.from_tensor_slices(ssl_images)
train_ds = (
    train_ds
    .map(parse_images, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    .shuffle(1024)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE)
)

# train network
epoch_wise_loss  = train_simclr(encoder_with_projection_head, train_ds, optimizer, criterion,
                 temperature=0.1, epochs=200)

plt.figure()
plt.plot(epoch_wise_loss)
plt.show()

# Save model
model_name = 'model_name'
encoder_path = '/content/drive/MyDrive/Tese/code/Models/Self_supervised/' + model_name
encoder.save(encoder_path)

Barlow Twins

In [10]:
#------------------Model--------------#
def encoderModel(size):
  model = tf.keras.Sequential()
  #------Encoder------#
  model.add(layers.Input(shape=(224,224,3)))
  model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
  model.add(layers.AveragePooling2D((2, 2), padding='same'))
  model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
  model.add(layers.AveragePooling2D((2, 2), padding='same'))
  model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
  model.add(layers.AveragePooling2D((2, 2), padding='same'))
  model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
  model.add(layers.AveragePooling2D((2, 2), padding='same'))
  model.add(layers.Conv2D(64, (3, 3), activation='tanh', padding='same'))
  model.add(layers.AveragePooling2D((2, 2), padding='same'))



  model.add(layers.Flatten())
  model.add(layers.Dense(size))
  model.add(layers.BatchNormalization())
  model.add(layers.Activation("relu"))
  model.add(layers.Dense(size))
  model.add(layers.Activation("relu"))
  model.add(layers.BatchNormalization())
  model.add(layers.Dense(size))

  return model

In [11]:
#--------Data Augmentation---------#
def random_resize_crop(image, crop_size=224):
    size = np.random.randint(100,200)
    image = tf.image.random_crop(image, [size,size,3])
    image = tf.image.resize(image,[crop_size,crop_size])
    return image

def flip_random_crop(image):
    image = tf.image.random_flip_left_right(image)
    image = random_resize_crop(image)
    return image


def color_jitter(x, strength=1):
    x = tf.image.random_saturation(x, lower=1 - 0.3 * strength, upper=1 + 0.3 * strength)
    x = tf.image.random_hue(x, max_delta=0.2 * strength)
    x = tf.clip_by_value(x, 0, 1)
    return x


def color_drop(x):
    x = tf.image.rgb_to_grayscale(x)
    x = tf.tile(x, [1, 1, 3])
    return x

def _rotatedRectWithMaxArea( w, h, angle):
      """
      Given a rectangle of size wxh that has been rotated by 'angle' (in
      radians), computes the width and height of the largest possible
      axis-aligned rectangle (maximal area) within the rotated rectangle.
      """
      if w <= 0 or h <= 0:
        return 0,0

      width_is_longer = w >= h
      side_long, side_short = (w,h) if width_is_longer else (h,w)

      # since the solutions for angle, -angle and 180-angle are all the same,
      # if suffices to look at the first quadrant and the absolute values of sin,cos:
      sin_a, cos_a = abs(math.sin(angle)), abs(math.cos(angle))
      if side_short <= 2.*sin_a*cos_a*side_long or abs(sin_a-cos_a) < 1e-10:
        # half constrained case: two crop corners touch the longer side,
        #   the other two corners are on the mid-line parallel to the longer line
        x = 0.5*side_short
        wr,hr = (x/sin_a,x/cos_a) if width_is_longer else (x/cos_a,x/sin_a)
      else:
        # fully constrained case: crop touches all 4 sides
        cos_2a = cos_a*cos_a - sin_a*sin_a
        wr,hr = (w*cos_a - h*sin_a)/cos_2a, (h*cos_a - w*sin_a)/cos_2a

      return wr,hr

def random_rotation(x):
    imgs = []
    for i in x:
        angle = np.random.randint(1,20)
        wr, hr = _rotatedRectWithMaxArea(224,224,math.radians(angle))
        # Use PIL or other library of the sort to rotate
        i = imutils.rotate(np.array(i), angle=angle)
        i = tf.image.central_crop(i, float(hr)/224) 
        i = tf.image.resize(i, [224,224])
        imgs.append(i)
    return np.array(imgs)


def random_apply(func, x, p):
    if tf.random.uniform([], minval=0, maxval=1) < p:
        return func(x)
    else:
        return x

def custom_augment1(image):
    image = tf.cast(image, tf.float32)
    image = flip_random_crop(image)
    image = random_apply(color_jitter, image, p=1)
    return image

def custom_augment2(image):
    image = tf.cast(image, tf.float32)
    image = flip_random_crop(image) 
    image = random_apply(color_jitter, image, p=0.7)
    return image

In [ ]:
!git clone https://github.com/sayakpaul/Barlow-Twins-TF
%cd Barlow-Twins-TF

In [13]:
#Aux variables
AUTO = tf.data.AUTOTUNE
BATCH_SIZE = 128
EPOCHS = 75
SEED = 42

STEPS_PER_EPOCH = len(ssl_images) // BATCH_SIZE
TOTAL_STEPS = STEPS_PER_EPOCH * EPOCHS
WARMUP_EPOCHS = int(EPOCHS * 0.1)
WARMUP_STEPS = int(WARMUP_EPOCHS * STEPS_PER_EPOCH)

import lr_scheduler

lr_decayed_fn = lr_scheduler.WarmUpCosine(
    learning_rate_base=1e-3,
    total_steps=EPOCHS * STEPS_PER_EPOCH,
    warmup_learning_rate=0.0,
    warmup_steps=WARMUP_STEPS
)

#Get transformations to apply to images
def get_ssl_ds(dataset, rotA=False, rotB=False):
  """
    datset: images to train during ssl phase
    rotA and rotB: flags that indicate if rotation transformation should be applied              
  """
  #get to copies of the same image to separate into two diferent sets 
  #the transformation applied to each set may not be the same
  datasetA, datasetB = dataset, dataset
  if rotA:
    datasetA = random_apply(random_rotation,dataset, p=1)  
  if rotB:
    datasetB = random_apply(random_rotation,dataset, p=1)

  ssl_ds_A = tf.data.Dataset.from_tensor_slices(datasetA)
  ssl_ds_A = (
      ssl_ds_A.shuffle(1024, seed=SEED)
      .map(custom_augment1, num_parallel_calls=AUTO)
      .batch(BATCH_SIZE)
      .prefetch(AUTO)
  )
  
  
  ssl_ds_B = tf.data.Dataset.from_tensor_slices(datasetB)
  ssl_ds_B = (
      ssl_ds_B.shuffle(1024, seed=SEED)
      .map(custom_augment2, num_parallel_calls=AUTO)
      .batch(BATCH_SIZE)
      .prefetch(AUTO)
  )

  # We then zip both of these datasets.
  ssl_ds = tf.data.Dataset.zip((ssl_ds_A, ssl_ds_B))
  return ssl_ds

def off_diagonal(x):
    n = tf.shape(x)[0]
    flattened = tf.reshape(x, [-1])[:-1]
    off_diagonals = tf.reshape(flattened, (n-1, n+1))[:, 1:]
    return tf.reshape(off_diagonals, [-1])


def normalize_repr(z):
    z_norm = (z - tf.reduce_mean(z, axis=0)) / tf.math.reduce_std(z, axis=0)
    return z_norm


def compute_loss(z_a, z_b, lambd):
    # Get batch size and representation dimension.
    batch_size = tf.cast(tf.shape(z_a)[0], z_a.dtype)
    repr_dim = tf.shape(z_a)[1]


    # Normalize the representations along the batch dimension.
    z_a_norm = normalize_repr(z_a)
    z_b_norm = normalize_repr(z_b)

    # Cross-correlation matrix.
    c = tf.matmul(z_a_norm, z_b_norm, transpose_a=True) / batch_size

    # Loss.
    on_diag = (1) - tf.linalg.diag_part(c) 
    on_diag = tf.reduce_sum(tf.pow(on_diag, 2))
    off_diag = off_diagonal(c)
    off_diag = tf.reduce_sum(tf.pow(off_diag, 2))
    loss = on_diag + (lambd * off_diag)
    return loss 


In [14]:
class BarlowTwins(tf.keras.Model):
    def __init__(self, encoder, lambd=5e-3):
        super(BarlowTwins, self).__init__()
        self.encoder = encoder
        self.lambd = lambd
        self.loss_tracker = tf.keras.metrics.Mean(name="loss")

    @property
    def metrics(self):
        return [self.loss_tracker]

    def train_step(self, data):
        # Unpack the data.
        ds_one, ds_two = data

        # Forward pass through the encoder and predictor.
        with tf.GradientTape() as tape:
            z_a, z_b = self.encoder(ds_one, training=True), self.encoder(ds_two, training=True)
            loss = compute_loss(z_a, z_b, self.lambd) 

        # Compute gradients and update the parameters.
        gradients = tape.gradient(loss, self.encoder.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.encoder.trainable_variables))

        # Monitor loss.
        self.loss_tracker.update_state(loss)
        return {"loss": self.loss_tracker.result()}

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_decayed_fn)

#Get model
encoder = encoderModel(1024)
barlow_twins = BarlowTwins(encoder)
barlow_twins.compile(optimizer=optimizer)

#apply transformations to the datasey
ssl_ds = get_ssl_ds(ssl_images)

# train network
history = barlow_twins.fit(ssl_ds,
                epochs=EPOCHS,
                batch_size=128,
                shuffle=True)


# Save the model
model_name = 'model_name'
Pmodel = '/content/drive/MyDrive/Tese/code/Models/Self_supervised/' + model_name
barlow_twins.encoder.save(Pmodel)

### **Supervised Phase**

In [9]:
#Create pretrain model
def create_classifier(encoder, trainable=True):
    for layer in encoder.layers:
        layer.trainable = trainable

    inputs = tf.keras.Input(shape=(224,224,3))
    features = encoder(inputs)
    features = layers.GlobalAveragePooling2D()(features)
    outputs = layers.Dense(1, activation="sigmoid")(features)
    model = tf.keras.Model(inputs=inputs, outputs=outputs, name="fire-classifier")
    model.compile(
        optimizer='adam',
        loss=losses.BinaryCrossentropy(),
        metrics=['accuracy'],
    )
    model.summary()
    return model

In [10]:
#Model for baseline (all layers are randomly initialized)
def create_model():
    inputs = tf.keras.Input(shape=(224,224,3))
    features = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    features = layers.AveragePooling2D((2, 2), padding='same')(features)
    features = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(features)
    features = layers.AveragePooling2D((2, 2), padding='same')(features)
    features = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(features)
    features = layers.AveragePooling2D((2, 2), padding='same')(features)
    features = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(features)
    features = layers.AveragePooling2D((2, 2), padding='same')(features)
    features = layers.Conv2D(64, (3, 3), activation='tanh', padding='same')(features)
    features = layers.AveragePooling2D((2, 2), padding='same')(features)
    
    features = layers.GlobalAveragePooling2D()(features)
    outputs = layers.Dense(1, activation="sigmoid")(features)
    model = tf.keras.Model(inputs=inputs, outputs=outputs, name="fire-classifier")
    model.compile(
        optimizer='adam',
        loss=losses.BinaryCrossentropy(),
        metrics=['accuracy'],
    )
    return model

In [12]:
def train_evaluate_supervised(task, model_name, model_path, path_to_save, mode = True):
  # Define per-fold score containers
  acc_per_fold = []
  loss_per_fold = []
  f1score_per_fold = []
  avg_per_fold = []


  # Merge inputs and targets
  inputs = np.array(sup_images)
  targets = np.array(sup_labels)


  num_folds = 10
  # Define the K-fold Cross Validator
  kfold = KFold(n_splits=num_folds, shuffle=True)
  # K-fold Cross Validation model evaluation
  fold_no = 1

  #aux variables 
  true_labs =[]
  predict_labs = []
  predict_orig_labs = []


  path_name = model_path + model_name
  #----------TRAIN SUPERVISED LAYERS-----------------#
  for train, test in kfold.split(inputs, targets):
    
    model = load_model(path_name)
    #For barlow twins and autoencoder it's necessary to remove some layers (simclr only saves encoder component) 
    #remove 6 layers for autoencoder and 8 for barlow twins
    if task == 'autoencoder':
      model.pop()
      model.pop()
      model.pop()
      model.pop()
      model.pop()
      model.pop()
    elif task == 'BT':
      model.pop()
      model.pop()
      model.pop()
      model.pop()
      model.pop()
      model.pop()
      model.pop()  
      model.pop() 


    if task == 'Baseline':
      classifier = create_model()

    #If training and evaluating pretrained model  
    else:
      encoder = tf.keras.Sequential()
      for layer in model.layers:
        encoder.add(layer)
      
      classifier = create_classifier(encoder, trainable=mode)


    #get the racio between positive and negative examples
    pos = sum(targets[train])
    w_pos = len(targets[train])/(pos*2)
    w_neg = len(targets[train])/((len(targets[train])-pos)*2)
    class_weights = {0: w_neg, 1:w_pos}
    # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')

    # Fit data to model
    history = classifier.fit(inputs[train], targets[train], batch_size=32, epochs=75, class_weight=class_weights, callbacks=[])

    # Generate generalization metrics
    scores = classifier.evaluate(inputs[test], targets[test], verbose=0)
    print(f'Score for fold {fold_no}: {classifier.metrics_names[0]} of {scores[0]}; {classifier.metrics_names[1]} of {scores[1]*100}%')
    
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])
    predict_orig = classifier.predict(inputs[test])
    predict = tf.round(predict_orig)
    if len(predict_labs) == 0:
      true_labs = targets[test]
      predict_labs = predict
      predict_orig_labs = predict_orig 
    else:
      predict_orig_labs = np.concatenate((predict_orig_labs, predict_orig), axis=0)
      predict_labs = np.concatenate((predict_labs, predict), axis=0)
      true_labs = np.concatenate((true_labs, targets[test]), axis=0)
    

    
    print('------------------------------------------------------------------------')
    matrix = confusion_matrix(targets[test], predict)
    print(f'> Confusion matrix: {matrix}')
    print('------------------------------------------------------------------------')
    f1score_per_fold.append(f1_score(targets[test], predict))
    print(f'> F1 score: {f1score_per_fold[fold_no-1]}')
    print('------------------------------------------------------------------------')
    avg_per_fold.append(average_precision_score(targets[test], predict_orig))
    print('------------------------------------------------------------------------')
    print(f'> Avg precision: {avg_per_fold[fold_no-1]}')
    print('------------------------------------------------------------------------')

    plt.figure()
    plt.plot(history.history['loss'])
    plt.title("Loss")
    plt.show()
    plt.figure()
    plt.plot(history.history['accuracy'])
    plt.title("Acc")
    plt.show()

    
    # Increase fold number
    fold_no = fold_no + 1


  # == Provide average scores ==
  print('------------------------------------------------------------------------')
  print('Score per fold')
  for i in range(0, len(acc_per_fold)):
    print('------------------------------------------------------------------------')
    print(f'> Fold {i+1} -  Accuracy: {acc_per_fold[i]}%  - F1score: {f1score_per_fold[i]}')
  print('------------------------------------------------------------------------')
  print('Average scores for all folds:')
  print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
  print(f'> Loss: {np.mean(loss_per_fold)} (+- {np.std(loss_per_fold)})')
  print(f'> F1score: {np.mean(f1score_per_fold)} (+- {np.std(f1score_per_fold)})')
  print(f'> Avg precision: {np.mean(avg_per_fold)} (+- {np.std(avg_per_fold)})')
  print('------------------------------------------------------------------------')
  print('------------------------------------------------------------------------')
  print(f'> Median: {median(acc_per_fold)}')
  print('------------------------------------------------------------------------')
  print('------------------------------------------------------------------------')
  print('Confusion Matrix:')
  matrix = confusion_matrix(true_labs, predict_labs)
  print(f'> Confusion matrix: {matrix}')
  print('------------------------------------------------------------------------')
  print('F1 score:')
  f1score = f1_score(true_labs, predict_labs)

  print(f'> F1 score: {f1score}')
  print('------------------------------------------------------------------------')
  print('------------------------------------------------------------------------')
  avg_precision_final = average_precision_score(true_labs, predict_orig_labs)
  print(f'> Avg precision: {avg_precision_final}')
  print('------------------------------------------------------------------------')


  #save csv file with results
  import csv 
  folder_path = path_to_save + model_name + '.csv' 
  header = ['accuracy', 'f1score', 'avg_precision']
  with open(folder_path,'w') as fd:
      writer = csv.writer(fd)
      # write the header
      writer.writerow(header)
      for i in range(len(acc_per_fold)):
        row = [acc_per_fold[i], f1score_per_fold[i], avg_per_fold[i]]
        writer.writerow(row)

In [ ]:
model_name = 'model_name'
model_path = '/content/drive/MyDrive/Tese/code/Models/Self_supervised/'
path_to_save = '/content/drive/MyDrive/Tese/code/results/' 
task = 'SimCLR'  #options: [autoencoder, SimCLR, BT, Baseline]

train_evaluate_supervised(task, model_name, model_path, path_to_save, True)